In [1]:
import sys
import os
import numpy as np
from PIL import Image
from flax.training import checkpoints
import traceback
from jaxrl_m.vision import encoders
from jaxrl_m.agents import agents
from jaxrl_m.data.bridge_dataset import multi_embed
from jaxrl_m.data.language import load_mapping, lang_encode
from jaxrl_m.vision.clip import process_image, process_text
from absl import app, flags, logging

import time
from datetime import datetime
from experiments.kevin.configs.bridgedata_config import get_config
import jax
import time
import tensorflow as tf
import jax.numpy as jnp
from flax.core import FrozenDict

import nest_asyncio
import json
nest_asyncio.apply()

FIXED_STD = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

def unnormalize_action(action, mean, std):
    return action * std + mean

/nfs/nfs1/users/vmyers/miniconda3/lib/python3.8/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(
2023-08-18 05:09:36.614347: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-08-18 05:09:38.429470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-08-18 05:09:38.429597: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror:

In [2]:
NAME = "all_multimodal_lawrence_lcbc_20230817_002757"
STEP = 10_000
wandb_run = f"widowx-gcrl/jaxrl_m_bridgedata/{NAME}"
checkpoint_path = f"checkpoint_{STEP}"

#api = wandb.Api()
#run = api.run(wandb_run)
class run: config = json.load(open("run_config_ur5.json", "rb")) # use cached config to avoid wandb login

encoder_def = encoders[run.config["encoder"]](**run.config["encoder_kwargs"])
task_encoder_defs = {
    k: encoders[run.config["task_encoders"][k]](
        **run.config["task_encoder_kwargs"][k]
    )
    for k in ("image", "language")
    if k in run.config["task_encoders"]
}

action_metadata = run.config["bridgedata_config"]["action_metadata"]
action_mean = np.array(action_metadata["mean"])
action_std = np.array(action_metadata["std"])

In [3]:
load_mapping(run.config["data_path"])

example_batch = {
    "observations": {"image": jnp.zeros((10, 128, 128, 3), dtype=jnp.uint8)},
    "initial_obs": {"image": jnp.zeros((10, 128, 128, 3), dtype=jnp.uint8)},
    "goals": {
        "image": jnp.zeros((10, 128, 128, 3), dtype=jnp.uint8),
        "language": jnp.zeros((10, 512)),
        "language_mask": jnp.ones(10, dtype=bool),
    },
    "actions": jnp.zeros((10, 7), dtype=jnp.float32),
}

rng = jax.random.PRNGKey(0)
rng, construct_rng = jax.random.split(rng)
agent = agents[run.config["agent"]].create(
    rng=construct_rng,
    observations=example_batch["observations"],
    initial_obs=example_batch["initial_obs"],
    goals=example_batch["goals"],
    actions=example_batch["actions"],
    encoder_def=encoder_def,
    task_encoder_defs=task_encoder_defs,
    **run.config["agent_kwargs"],
)
restored = checkpoints.restore_checkpoint(checkpoint_path, agent)
assert restored is not agent
agent = restored

optimizer partitions:
FrozenDict({
    contrastive_temp: 'encoder',
    encoders_actor_0: {
        Dense_0: {
            bias: 'actor',
        },
    },
    networks_actor: {
        MLP_0: {
            layers_2: {
                bias: 'actor',
                kernel: 'actor',
            },
        },
    },
})


In [4]:
val_data = jnp.load('ur5_val/out.npy', allow_pickle=True)

def get_trans(idx, time):
    return (
        val_data[idx]["observations"][0]["images0"],
        val_data[idx]["observations"][time]["images0"], 
        val_data[idx]['language'][time].item(),
    )

In [5]:
def predict_actions(initial_obs, current_obs, language):
    return agent.sample_actions(
        dict(image=current_obs),
        dict(language=multi_embed(language)),
        dict(image=initial_obs),
        seed=rng,
        modality="language",
        argmax=True,
    )

In [6]:
predict_actions(*get_trans(idx=0, time=10))

Array([-0.7309984 ,  0.58064705,  0.83200514,  0.10265408, -0.04553585,
        0.04656164,  0.01063498], dtype=float32)